# 基于 ERNIE-4.5-0.3B 的大语言模型监督微调 (SFT) 教程

## 1. 引言

欢迎来到这篇关于使用 ERNIEkit 对 ERNIE-4.5-0.3B 模型进行监督微调 (Supervised Fine-Tuning, SFT) 的教程！在上一篇预训练教程中，我们已经探索了如何从零开始构建一个语言模型，赋予它理解和生成文本的基础能力。然而，一个经过预训练的通用大模型，就像一个知识渊博但未经专门训练的通才，它拥有海量的知识，却不一定能精准地遵循我们的具体指令来完成特定任务。

### 1.1 什么是大语言模型监督微调 (SFT)？

**监督微调 (SFT)** 是将通用大模型转化为专业助手的关键步骤。它在预训练好的模型基础上，利用一个包含“指令 (Instruction/Prompt)”和“期望输出 (Response/Output)”的标注数据集，对模型进行进一步的训练。这个过程好比是为一位博学的通才提供“岗前培训”，教会它如何理解并遵循人类的指令，如何针对特定问题生成高质量的回答，或者如何模仿某种特定的对话风格和格式。

**SFT 与预训练 (PT) 的核心区别：**

| 特性 | 预训练 (PT) | 监督微调 (SFT) |
|---|---|---|
| **目标** | 学习通用的语言规律、语法结构和世界知识 | 学习遵循指令、解决特定任务、对齐人类意图和价值观 |
| **数据** | 大规模、**无标签**的纯文本数据（如网页、书籍、代码） | 相对较小规模、**有高质量标签**的“指令-输出”对数据 |
| **学习方式** | 无监督或自监督学习（如预测下一个词） | 有监督学习（学习从输入指令到期望输出的映射） |
| **计算资源** | 通常需求巨大，需要海量数据和超长训练时间 | 相对较小，可在预训练模型基础上进行高效、快速的迭代 |
| **模型角色** | 训练一个“基础模型” (Base Model)，奠定能力基础 | 在“基础模型”上进行“指令调优” (Instruction Tuning)，特化模型行为 |

简而言之，预训练让模型“学会说话”，而监督微调则教模型“好好说话”、“听话办事”，使其行为与人类的期望对齐。

### 1.2 SFT 的应用场景和意义

SFT 是释放大语言模型潜力的核心技术，其意义和应用场景极为广泛。我们可以通过下表来直观感受：

|  | 应用场景 | 核心意义 | 
| :---: | :--- | :--- | 
| 🎯 | **提升特定任务性能** | 通过在特定领域的指令数据上进行SFT，可以显著提升模型在客服问答、代码生成、法律文书摘要、医疗咨询等任务上的专业性、准确性和可靠性。 | 
| 🙏 | **模型对齐 (Alignment)** | SFT 是实现模型与人类意图和价值观对齐的关键一步。通过精心设计的指令和期望的回答（通常强调有用性、真实性和无害性），引导模型生成负责任、有帮助的内容。 | 
| 💬 | **构建强大的对话式AI** | 所有顶尖的聊天机器人，无一不经过了大量的SFT训练。这使得它们能够进行流畅、自然、且富有逻辑的多轮对话，真正成为用户的得力助手。 | 
| 🎭 | **遵循特定风格或格式** | 我们可以通过SFT训练模型以固定的格式（如JSON、Markdown）输出，或者扮演特定的角色（如莎士比亚、技术专家），甚至生成具有特定情感基调的文本。 | 

### 1.3 为什么选择 ERNIE-4.5-0.3B 进行 SFT？

选择 `ERNIE-4.5-0.3B` 作为我们SFT实践的主角，是基于以下几点关键优势：

|  | 优势 | 详细说明 |
| :---: | :--- | :--- |
| 🏆 | **卓越的预训练基础** | `ERNIE-4.5-0.3B` 作为文心大模型的最新成员，其本身经过了海量、高质量数据的充分预训练，具备了世界领先的中文处理能力和通用语言理解能力。这是一个极其强大的SFT起点。 |
| 💰 | **参数规模适中，性价比高** | 0.3B（3亿）的参数量，使得在消费级或主流的计算资源上进行全参数SFT成为可能。这极大地降低了开发者和研究者进行实验和快速迭代的门槛，是探索SFT的最佳选择之一。 |
| 🛠️ | **ERNIEkit 的全面支持** | ERNIEkit 是飞桨为文心大模型量身打造的开发套件，提供了从预训练、SFT、到推理部署的全流程工具。其SFT脚本经过精心优化，支持全参数微调、LoRA等多种参数高效微调（PEFT）方法，并且配置灵活，使用便捷。 |

### 1.4 本教程的目标读者和学习成果

本教程旨在帮助不同背景的您，深入SFT的世界。看看您属于哪一类读者，以及您将收获什么：

|  | 目标读者 | 您将获得的学习成果 (You Will Be Able To) |
| :---: | :--- | :--- |
| 🧑‍🎓 | **进阶学习者** | 深刻理解SFT的核心原理、应用价值及其与预训练的本质区别。 |
| 👨‍💻 | **开发者/工程师** | 掌握使用 ERNIEkit 对 ERNIE-4.5-0.3B 模型进行SFT的完整流程和最佳实践。 |
| 🔬 | **研究者** | 了解SFT数据的标准格式和构建方法，并对参数高效微调 (PEFT) 如LoRA有一个初步的认识。 |
| 🚀 | **所有读者** | 学会如何解读和配置 ERNIEkit 的SFT `yaml` 文件，启动训练任务，并分析训练过程，最终对微调后的模型进行推理测试，直观感受SFT带来的能力提升。 |

现在，让我们一起踏上征程，通过SFT技术，将强大的 ERNIE-4.5-0.3B 模型“调教”成更懂你、更能干的专属AI助手！

## 2. 环境准备

进行监督微调 (SFT) 的第一步是搭建一个稳定、高效的开发环境。这包括安装核心的深度学习框架 PaddlePaddle、模型开发套件 ERNIEkit，并准备好我们本次实践的主角——ERNIE-4.5-0.3B 模型。

### 2.1 安装 PaddlePaddle 和 aistudio-sdk

确保您安装了最新或推荐版本的 PaddlePaddle。推荐使用GPU版本的 PaddlePaddle 以获得极致的训练速度。同时，我们需要安装 `aistudio-sdk`，它是一个非常方便的工具，可以帮助我们轻松下载AI-Studio上的模型资源。

*在AI studio上运行本项目的同学不需要运行下方的环境安装代码块*

In [ ]:
# 确保 pip 是最新版本
!python -m pip install --upgrade pip

# 安装 PaddlePaddle (GPU 版本, 推荐使用最新的稳定版)
# 如果您的环境（如CUDA版本）不同，请访问官网获取对应指令: https://www.paddlepaddle.org.cn/install/quick
!python -m pip install paddlepaddle-gpu -i https://mirror.baidu.com/pypi/simple

# 安装 aistudio-sdk
!pip install --upgrade aistudio-sdk

**验证安装**：

In [1]:
import paddle

print(f"PaddlePaddle Version: {paddle.__version__}")

# 检查GPU是否可用
try:
    paddle.utils.run_check()
    if paddle.device.cuda.device_count() > 0:
        print(f"PaddlePaddle GPU is available! Found {paddle.device.cuda.device_count()} GPU(s).")
    else:
        print("PaddlePaddle GPU check passed, but no GPU found. Will use CPU.")
except Exception as e:
    print(f"PaddlePaddle GPU check failed: {e}")
    print("If you intended to use GPU, please check your CUDA setup and PaddlePaddle installation.")

/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/paddle/utils/cpp_extension/extension_utils.py:715: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)


PaddlePaddle Version: 3.1.0
Running verify PaddlePaddle program ... 
PaddlePaddle works well on 1 GPU.
PaddlePaddle is installed successfully! Let's start deep learning with PaddlePaddle now.
PaddlePaddle GPU is available! Found 1 GPU(s).


I0717 18:57:50.384768   265 pir_interpreter.cc:1524] New Executor is Running ...
W0717 18:57:50.386147   265 gpu_resources.cc:114] Please NOTE: device: 0, GPU Compute Capability: 8.0, Driver API Version: 12.8, Runtime API Version: 12.6
I0717 18:57:50.386685   265 pir_interpreter.cc:1547] pir interpreter is running by multi-thread mode ...


### 2.2 下载 ERNIEkit 仓库代码和 ERNIE-4.5-0.3B 模型

ERNIEkit 的SFT脚本 (`train.py`) 和相关的模型、训练配置文件都位于其官方仓库中。同时，我们需要使用 `aistudio` 命令行工具下载 ERNIE-4.5-0.3B 模型权重。

*在AI studio运行的同学不需要运行下方代码块*

- ERNIE：/home/aistudio/ERNIE-develop.zip
- model：/home/aistudio/data/models/30654/ERNIE-4.5-0.3B-Base-Paddle

In [ ]:
# 1. 克隆 ERNIEkit 仓库
# 我们将克隆最新的 ERNIE-develop 分支，以获取最新的功能和优化
!git clone https://github.com/PaddlePaddle/ERNIE.git -b develop ERNIE-develop

# 2. 下载 ERNIE-4.5-0.3B 模型权重
# 使用 aistudio-sdk 提供的命令行工具下载
# 模型将被下载到 baidu/ERNIE-4.5-0.3B-Paddle 目录下
!aistudio download --model PaddlePaddle/ERNIE-4.5-0.3B-Paddle --local_dir baidu/ERNIE-4.5-0.3B-Paddle

**重要目录结构说明**：

执行完上述命令后，您的工作目录下应该有类似这样的结构：

```
.
├── ERNIE-develop/
│   ├── examples/
│   │   ├── configs/
│   │   │   └── ERNIE-4.5-0.3B/
│   │   │       └── sft/
│   │   │           └── run_sft_8k.yaml  <-- 这是我们的SFT配置文件
│   │   └── data/
│   │       └── sft-train.jsonl        <-- 这是示例的SFT数据
│   ├── train.py                     <-- 这是ERNIEkit的训练脚本
│   └── ... (其他ERNIEkit代码)
└── baidu/
    └── ERNIE-4.5-0.3B-Paddle/       <-- 这里存放着模型权重和配置文件
        ├── model_state.pdparams
        ├── tokenizer_config.json
        └── ... (其他模型文件)
```

Ai studio的同学模型目录则在data文件夹内！

在后续的教程中，我们将主要在 `ERNIE-develop` 目录下进行操作，并使用 `train.py` 脚本，结合 `examples/configs/ERNIE-4.5-0.3B/sft/run_sft_8k.yaml` 配置文件，来加载 `baidu/ERNIE-4.5-0.3B-Paddle` 目录下的模型进行微调。

请确保您的目录结构与此一致，以便顺利进行后续步骤。

环境和资源都已准备就绪，接下来，让我们深入了解SFT的核心——数据。


## 3. SFT 数据准备

“Garbage in, garbage out.” 这句名言在SFT中体现得淋漓尽致。SFT的效果在很大程度上取决于所用指令数据的质量。高质量的数据能够引导模型学习到正确的行为模式，而低质量的数据则可能导致模型产生有偏见、不准确甚至有害的输出。

## 3.1 SFT 数据的特点与格式

SFT数据由成对的“指令 (Instruction/Prompt)”和“理想输出 (Response/Output)”组成。ERNIEkit 使用 JSON Lines (jsonl) 格式，其中每一行都是一个独立的JSON对象，代表一个训练样本。这种格式清晰、可扩展，非常适合大规模数据集的处理。

**ERNIEkit SFT 数据格式详解：**

一个标准的 ERNIEkit SFT 样本结构如下：

```json
{"src": "你好", "tgt": "你好！很高兴为您服务。"}
{"src": "请写一首关于月亮的诗。", "tgt": "床前明月光，疑是地上霜。举头望明月，低头思故乡。"}
```

*   `src` (Source): 代表输入给模型的指令或问题。这部分内容会经过模板化处理，作为模型的输入提示 (Prompt)。
*   `tgt` (Target): 代表期望模型生成的回答。这是模型需要学习和模仿的目标输出。

**对话模板的重要性：**

像 ERNIE-4.5 这样的对话模型，在训练时通常会遵循一个特定的对话模板。这个模板定义了用户 (user) 和助手 (assistant) 的角色，并使用特殊的 token 来分隔不同的轮次。ERNIEkit 的训练脚本会自动将 `src` 和 `tgt` 包装成模型预训练时所使用的模板格式。例如，一个简化的模板可能如下所示：

```
<|im_start|>user
{src}<|im_end|>
<|im_start|>assistant
{tgt}<|im_end|>
```

模型在训练时，其任务就是根据 `user` 的内容，续写出 `assistant` 的部分。通过这种方式，模型学会了在对话中扮演助手的角色。

**高质量SFT数据的关键要素：**

*   **多样性**: 指令应覆盖尽可能多的主题和技能，如问答、创作、摘要、翻译、代码生成等。
*   **复杂性**: 指令应包含不同难度级别，从简单的事实问答到需要深度推理和创造力的复杂任务。
*   **准确性**: `tgt` 的内容必须是准确、高质量且无害的。
*   **一致性**: 输出的风格、格式和详细程度应保持一致，除非您有意训练模型掌握多种风格。


### 3.2 使用 ERNIEkit 提供的示例 SFT 数据集

ERNIEkit 在 `ERNIE-develop/examples/data/` 目录下提供了一个名为 `sft-train.jsonl` 的示例数据集。这是一个小型的、经过清洗的数据集，非常适合用于快速跑通SFT流程和功能验证。

让我们来查看一下这个示例数据的内容：

In [2]:
import json

# 定义示例数据路径
sample_data_path = "./ERNIE-develop/examples/data/sft-train.jsonl"

# 读取并打印前5条样本
print(f"查看示例数据: {sample_data_path}\n---")
with open(sample_data_path, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i >= 5:
            break
        data_sample = json.loads(line.strip())
        print(f"【样本 {i+1}】")
        print(f"  SRC: {data_sample['src']}")
        print(f"  TGT: {data_sample['tgt']}\n")

查看示例数据: ./ERNIE-develop/examples/data/sft-train.jsonl
---
【样本 1】
  SRC: ['我们如何在日常生活中减少用水？']
  TGT: ['1. 使用节水装置，如节水淋浴喷头和水龙头。 \n2. 使用水箱或水桶收集家庭废水，例如洗碗和洗浴。 \n3. 在社区中提高节水意识。 \n4. 检查水管和灌溉系统的漏水情况，并及时修复它们。 \n5. 洗澡时间缩短，使用低流量淋浴头节约用水。 \n6. 收集雨水，用于园艺或其他非饮用目的。 \n7. 刷牙或擦手时关掉水龙头。 \n8. 减少浇水草坪的时间。 \n9. 尽可能多地重复使用灰水（来自洗衣机、浴室水槽和淋浴的水）。 \n10. 只购买能源效率高的洗碗机和洗衣机。']

【样本 2】
  SRC: ['政府可以采取哪些策略来减少空气污染？']
  TGT: ['1. 实施强制的车辆排放标准和基于激励的计划，以降低车辆的碳足迹。\n2. 增加公共交通工具，减少公众对车辆的依赖。\n3. 增加对空气污染的影响的认识，鼓励市民减少污染物的生成。\n4. 投资于可再生能源的研究和开发，如太阳能和风能。\n5. 在工厂和发电厂安装空气污染控制装置，例如洗涤器。\n6. 对车辆和工厂使用清洁燃料。\n7. 实施更好的城市规划和控制拓展。\n8. 改善农业效率，减少化肥和杀虫剂的使用。\n9. 种植更多的树木以减少空气污染。\n10. 减少木材、煤炭和生物质的燃烧。']

【样本 3】
  SRC: ['可再生能源的存在对环境有什么影响？']
  TGT: ['可再生能源的存在可以帮助减少空气污染和温室气体排放，因为它们几乎不会排放二氧化碳、二氧化硫等空气污染物。此外，使用可再生能源可以促进能源效率的进一步提高和能源利用的改善。可再生能源也可以帮助减少对化石燃料的依赖，这不仅可以减少排放，而且还可以帮助减少全球气候变化的风险。最后，可再生能源可以帮助保护自然资源，减少为了能源生产而开发更多土地和资源的需要。']

【样本 4】
  SRC: ['解释神经网络如何学习。']
  TGT: ['神经网络是一种机器学习算法，它使用连接的节点集合来近似可以将输入变量映射到输出的函数。为了学习神经网络的参数，计算机需要调整节点之间连接的权重，以便网络为给定输入产生正确的输出。这个调

### 3.3 如何准备自己的 SFT 数据集

在实际应用中，您通常需要根据自己的特定需求构建数据集。以下是构建自定义SFT数据集的一般步骤：

1.  **确定目标任务**: 明确您希望模型具备哪些能力。是用于垂直领域的智能客服，还是用于生成特定风格的营销文案？
2.  **收集或生成原始数据**: 
    *   **收集**: 从现有的文档、工单、FAQ、或用户日志中提取“问题-答案”对。
    *   **人工标注**: 聘请领域专家或标注人员，根据任务需求编写高质量的指令和回答。
    *   **模型生成**: 使用更强大的模型（如GPT-4）生成初步的指令和回答，然后由人工进行筛选和精炼。这是目前构建大规模SFT数据集的主流方法之一。
3.  **清洗和格式化**: 
    *   去除低质量、重复或不相关的数据。
    *   进行数据增强，例如，对同一个指令，可以有多种不同的问法。
    *   将数据整理成 ERNIEkit 所需的 `jsonl` 格式，即每行一个包含 `src` 和 `tgt` 键的JSON对象。
4.  **划分数据集**: 将数据集划分为训练集 (train) 和评估集 (eval/dev)。评估集用于在训练过程中监控模型性能，避免过拟合。通常比例为 95%:5% 或 98%:2%。

准备好高质量的数据是SFT成功的基石。现在我们已经了解了数据的准备工作，下一步将深入剖析 ERNIEkit 的SFT配置文件，了解如何通过它来掌控整个微调过程。

## 4. SFT 配置解析 (`run_sft_8k.yaml`)

ERNIEkit 的强大之处在于其灵活性和可配置性。所有的训练参数都集中在 `yaml` 配置文件中，通过修改这个文件，我们可以精确地控制SFT的每一个环节。让我们以 `ERNIE-develop/examples/configs/ERNIE-4.5-0.3B/sft/run_sft_8k.yaml` 为例，逐一解析其中的关键配置项。

### 4.1 数据配置 (Data)

这部分定义了训练和评估所使用的数据集。

```yaml
### data
train_dataset_type: "erniekit"
eval_dataset_type: "erniekit"
train_dataset_path: "./examples/data/sft-train.jsonl"
train_dataset_prob: "1.0"
eval_dataset_path: "./examples/data/sft-eval.jsonl"
eval_dataset_prob: "1.0"
max_seq_len: 8192
num_samples_each_epoch: 6000000
```

*   `train_dataset_type`, `eval_dataset_type`: 数据集格式类型，这里是 `erniekit`，表示使用我们之前讨论的 `{"src": ..., "tgt": ...}` 格式。
*   `train_dataset_path`, `eval_dataset_path`: 训练和评估数据集的路径。**这里您需要修改为您自己的数据路径。**
*   `train_dataset_prob`, `eval_dataset_prob`: 数据集采样概率。当有多个数据集时，可以设置不同概率进行混合训练。
*   `max_seq_len`: 模型处理的最大序列长度。`8192` (8K) 是 ERNIE-4.5 的一个常用设置，可以处理非常长的上下文。如果您的GPU显存有限，可以适当减小此值，但这可能会影响模型处理长文本的能力。
*   `num_samples_each_epoch`: 每个epoch中包含的样本数。这个值通常设置得比较大，以确保模型在每个epoch中都能看到足够多的数据。


### 4.2 模型配置 (Model)

这部分定义了要微调的基础模型及其相关设置。

```yaml
### model
model_name_or_path: baidu/ERNIE-4.5-0.3B-Paddle
fine_tuning: Full
fuse_rope: True
use_sparse_head_and_loss_fn: True
```

*   `model_name_or_path`: **核心参数**。指定基础模型的路径。这里我们使用之前下载的 `baidu/ERNIE-4.5-0.3B-Paddle`。
*   `fine_tuning`: 微调方式。`Full` 表示全参数微调，即更新模型的所有权重。这是最彻底但资源消耗也最大的方式。其他选项如 `lora` 将在后续介绍。
*   `fuse_rope`: 是否使用优化的 Rotary Position Embedding。`True` 可以提升计算效率。
*   `use_sparse_head_and_loss_fn`: 是否使用稀疏的头部和损失函数。这是针对特定模型结构的优化，保持默认即可。

### 4.3 微调核心配置 (Finetuning)

这是控制整个训练流程的核心部分。

```yaml
### finetuning
# base
stage: SFT
seed: 23
do_train: True
do_eval: True
distributed_dataloader: False
dataloader_num_workers: 1
batch_size: 1
num_train_epochs: 1
max_steps: 100
max_evaluate_steps: 10000
eval_steps: 10000
evaluation_strategy: steps
save_steps: 10000000
save_total_limit: 5
save_strategy: steps
logging_steps: 1
release_grads: True
gradient_accumulation_steps: 8
logging_dir: ./vdl_log
output_dir: ./output
disable_tqdm: True
```

*   `stage`: 训练阶段，这里是 `SFT`。
*   `seed`: 随机种子，用于保证实验的可复现性。
*   `do_train`, `do_eval`: 是否执行训练和评估。
*   `batch_size`: **重要**。每个设备上的批处理大小。由于大模型非常消耗显存，`1` 是一个常见的起始值。
*   `gradient_accumulation_steps`: **重要**。梯度累积步数。这是一种“虚拟”扩大batch size的技巧。实际的 `batch_size` 是 `batch_size` * `gradient_accumulation_steps` * `num_gpus`。在这里，等效的batch size是 `1 * 8 = 8`。增加此值可以模拟更大的batch size，有助于稳定训练，但会减慢训练速度。
*   `num_train_epochs`: 训练的总轮数。对于SFT，通常 `1` 到 `3` 个epoch就足够了。
*   `max_steps`: 最大训练步数。如果设置了此值，将覆盖 `num_train_epochs`。对于快速实验，可以设置为一个较小的值（如 `100`）。
*   `evaluation_strategy`, `eval_steps`: 评估策略。`steps` 表示每隔 `eval_steps` 步进行一次评估。
*   `save_strategy`, `save_steps`: 保存策略。`steps` 表示每隔 `save_steps` 步保存一个checkpoint。
*   `logging_steps`: 每隔多少步打印一次日志。
*   `output_dir`: 训练产物（如模型checkpoint）的输出目录。

### 4.4 训练与优化器配置 (Train & Optimizer)

这部分控制学习率、优化器等超参数。

```yaml
# train
warmup_steps: 20
learning_rate: 1.0e-5
lr_scheduler_type: cosine
min_lr: 1.0e-6

# optimizer
weight_decay: 0.1
adam_epsilon: 1.0e-8
adam_beta1: 0.9
adam_beta2: 0.95
offload_optim: True
```

*   `learning_rate`: **核心超参数**。学习率的大小直接影响模型的收敛速度和最终效果。`1.0e-5` 是一个适用于全参数微调的常用值。对于LoRA等PEFT方法，可能需要更大的学习率。
*   `lr_scheduler_type`: 学习率调度策略。`cosine` 表示使用余弦退火策略，是一种非常有效且常用的策略。
*   `warmup_steps`: 预热步数。在训练初期，学习率会从一个很小的值线性增长到设定的 `learning_rate`，这有助于训练的稳定。
*   `weight_decay`: 权重衰减，一种防止过拟合的正则化技术。
*   `offload_optim`: 是否将优化器状态卸载到CPU内存。这是一个节省显存的技巧，但会略微降低训练速度。


### 4.5 性能与并行配置 (Performance)

这部分用于配置分布式训练和混合精度训练，以提升效率和节省资源。


```yaml
# performance
tensor_parallel_degree: 1
pipeline_parallel_degree: 1
sharding_parallel_degree: 1
sharding: stage1
sequence_parallel: True
recompute: False
compute_type: bf16
fp16_opt_level: O2
```

*   `tensor_parallel_degree`, `pipeline_parallel_degree`, `sharding_parallel_degree`: 分布式训练的并行度。对于单卡训练，这些都应保持为 `1`。
*   `sharding`: ZeRO优化策略。`stage1` 可以在多卡训练时显著节省显存。
*   `sequence_parallel`: 序列并行，一种针对长序列的显存优化技术。
*   `recompute` (Gradient Checkpointing): **重要**。一种用计算换显存的技术。设置为 `True` 会在反向传播时重新计算前向传播的中间结果，而不是存储它们。这能大幅降低显存消耗，但会增加约20-30%的训练时间。**当显存不足时，开启此选项是首选**。
*   `compute_type`: 计算精度。`bf16` (BFloat16) 是一种专为深度学习设计的半精度浮点格式，它能大幅降低显存占用并加速计算，同时保持较好的数值稳定性。推荐在支持的硬件上（如NVIDIA A100/H100, RTX 30/40系列）使用。
*   `fp16_opt_level`: 混合精度训练等级。`O2` 是一个常用的设置，可以与 `bf16` 配合使用。

理解了这些配置项，您就拥有了自由定制SFT过程的能力。在下一节中，我们将把所有知识整合起来，正式启动SFT训练任务。

## 5. 启动 SFT 训练

理论和配置都已准备就绪，现在是时候亲手启动SFT训练，见证模型能力蜕变的过程了。ERNIEkit 通过一个简单的命令即可启动整个训练流程。

### 5.1 启动训练命令

我们将在 `ERNIE-develop` 目录下，使用 `train.py` 脚本，并指定我们的配置文件 `run_sft_8k.yaml`。

**在终端或命令行中执行：**

In [3]:
# 首先，请确保您位于 ERNIE-develop 目录下
%cd ./ERNIE-develop

/home/aistudio/ERNIE-develop


In [4]:
!erniekit train examples/configs/ERNIE-4.5-0.3B/sft/run_sft_8k.yaml

/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/paddle/utils/cpp_extension/extension_utils.py:715: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/paddle/utils/cpp_extension/extension_utils.py:715: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
LAUNCH INFO 2025-07-17 19:05:48,655 -----------  Configuration  ----------------------
LAUNCH INFO 2025-07-17 19:05:48,656 auto_cluster_config: 0
LAUNCH INFO 2025-07-17 19:05:48,656 auto_parallel_config: None
LAUNCH INFO 2025-07-17 19:05:48,656 auto_tuner_json: None
LAUNCH INFO 2025-07-17 19:05:48

### 5.2 解读训练日志

当训练开始后，您会看到大量的日志信息输出。学会解读这些日志是监控训练状态、判断模型是否正常收敛的关键。


让我们来解析其中的关键信息：

*   `global_step`: 当前的总训练步数。`10/100` 表示当前是第10步，总共需要训练100步（由 `max_steps` 定义）。
*   `epoch`: 当前的训练轮数。
*   `loss`: **最重要的指标**。它衡量了模型预测的输出与真实 `tgt` 之间的差距。**我们期望看到 `loss` 值随着训练的进行而稳步下降**。如果 `loss` 居高不下、剧烈震荡或者变为 `NaN`，都说明训练可能出了问题（例如，学习率过高、数据有问题等）。
*   `learning_rate`: 当前的学习率。您可以看到它在 `warmup_steps` 之后达到设定的值，然后根据 `lr_scheduler_type` 的策略进行变化。
*   `speed` / `ips`: 训练速度。分别表示每秒处理的步数和样本数。可以用来预估总的训练时间。
*   `eta`: 预计剩余训练时间 (Estimated Time of Arrival)。

**监控要点：**

1.  **Loss 是否下降**: 这是首要的健康指标。一个平稳下降的loss曲线是训练成功的标志。
2.  **GPU 利用率和显存**: 使用 `nvidia-smi` 命令可以实时查看GPU的工作状态。确保GPU利用率尽可能高，显存占用在合理范围内。如果显存溢出 (Out of Memory, OOM)，您需要减小 `batch_size`、`max_seq_len`，或者开启 `recompute`。
3.  **评估结果 (Eval Loss)**: 当达到 `eval_steps` 时，模型会在评估集上进行测试。我们同样期望 `eval_loss` 也能随之下降，这表明模型的泛化能力在提升。如果 `train_loss` 下降而 `eval_loss` 上升，说明模型可能发生了过拟合。

训练完成后，所有微调过的模型权重和配置文件都会保存在您 `yaml` 中 `output_dir` 指定的目录下。默认情况下，会有一个名为 `checkpoint-xxx` 的文件夹，其中 `xxx` 是保存时的步数。

## 6. 总结与展望

恭喜您！通过本教程的学习和实践，您已经成功掌握了使用 ERNIEkit 对 ERNIE-4.5-0.3B 大语言模型进行监督微调 (SFT) 的核心技术。我们从SFT的基本概念出发，一步步完成了环境准备、数据处理、配置解析、模型训练与推理评估的全过程。

### 6.1 本教程核心回顾

让我们再次梳理本次旅程的关键学习点：

1.  **理解了SFT的本质**：我们明确了SFT是在预训练模型基础上，通过有标签的“指令-输出”数据，教会模型如何遵循人类意图、完成特定任务的过程。它是连接通用大模型与具体应用场景的桥梁。
2.  **掌握了ERNIEkit SFT全流程**：我们学会了使用 ERNIEkit 这一强大的工具链，从下载模型、准备数据，到配置和启动训练，再到最终的推理验证，形成了一个完整的闭环。
3.  **精通了SFT配置**：我们深入解析了 `run_sft_8k.yaml` 配置文件中的各项参数，理解了如何通过调整学习率、batch size、序列长度、优化策略等来控制和优化SFT过程，并学会了如梯度累积、重计算 (Recompute)、混合精度等关键的显存优化技巧。
4.  **强调了数据的重要性**：我们认识到，高质量、多样化的SFT数据是决定微调成败的基石，并学习了如何构建符合ERNIEkit要求的数据集。

通过SFT，我们赋予了 ERNIE-4.5-0.3B 模型全新的、定制化的能力，使其从一个“博学的通才”向“专业的助手”迈出了坚实的一步。

### 6.2 后续学习与探索方向

SFT为我们打开了通往定制化大模型世界的大门，但这仅仅是一个开始。大语言模型的技术栈仍在飞速发展，以下是一些值得您进一步探索的方向：

*   **参数高效微调 (PEFT - Parameter-Efficient Fine-Tuning)** ：
    *   **LoRA (Low-Rank Adaptation)** ：全参数微调虽然效果好，但资源消耗大，且会为每个任务都保存一个完整的模型副本。LoRA是一种PEFT方法，它通过在模型的某些层中注入小的、可训练的“适配器”矩阵，冻结原始的大部分权重。这样，我们只需要训练和存储极少部分（通常不到1%）的参数，就能达到接近全参数微调的效果。ERNIEkit 对 LoRA 提供了良好的支持，您只需在配置文件中将 `fine_tuning: Full` 修改为 `fine_tuning: lora` 并配置相关参数即可尝试。

*   **从人类反馈中强化学习 (RLHF - Reinforcement Learning from Human Feedback)** ：
    *   SFT教会了模型“听话”，但如何让模型的回答更符合人类的偏好（例如，更有用、更无害、更有趣）？RLHF就是解决这个问题的关键技术。它通过收集人类对模型不同回答的偏好排序数据，训练一个“奖励模型”，然后使用强化学习算法（如PPO）来优化语言模型，使其生成的内容能获得更高的奖励分数。这是通往更高级、更对齐的AI的必经之路。

*   **更先进的模型与技术** ：
    *   持续关注文心大模型和ERNIEkit的最新进展，探索更大参数规模的模型、更先进的算法和更高效的训练框架。

*   **模型部署与服务化** ：
    *   将您微调好的模型部署成在线服务，是实现其价值的最后一公里。可以探索如Paddle Serving、FastDeploy等工具，学习如何进行模型量化、压缩，并提供高性能的推理API。

大语言模型的时代充满了无限的机遇和挑战。希望本教程能成为您探索这个激动人心领域的坚实起点。持续学习，不断实践，您将能够构建出越来越强大的AI应用。祝您在AI的探索之路上行稳致远！

# 问题反馈/与我联系： Wechat：G_Fuji